In [65]:
import openml
from fairlib import DataFrame
from fairlib.preprocessing.lfr import LFR  # Aggiornato il percorso dell'import
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
import numpy as np
import torch

In [66]:
def prepare_dataset():
    """
    Load and preprocess the dataset.
    """
    dataset = openml.datasets.get_dataset(179)
    X, y, _, names = dataset.get_data(target=dataset.default_target_attribute)

    for col in X.columns:
        if X[col].dtype.name == 'category':
            X[col] = X[col].astype(object)

        # Ora che è di tipo oggetto, possiamo riempire i valori mancanti in sicurezza
        if X[col].isna().any():
            X[col] = X[col].fillna("missing")

        le = LabelEncoder()
        X[col] = le.fit_transform(X[col])

    X = DataFrame(X)
    X = X.drop(columns=["fnlwgt"])
    y = y.apply(lambda x: x == ">50K").astype(int)

    return X, y

In [67]:
def evaluate_fairness(X_test, y_pred):
    """
    Evaluate the fairness metrics (SPD and DI) of the predictions.
    """
    X_test = X_test.copy()
    X_test["income"] = y_pred
    dataset = DataFrame(X_test)
    dataset.targets = "income"
    dataset.sensitive = "sex"

    spd = dataset.statistical_parity_difference()
    di = dataset.disparate_impact()
    return spd, di

In [68]:
def train_classifier(X_train, y_train):
    """
    Train a logistic regression classifier.
    """
    clf = LogisticRegression(random_state=42, max_iter=1000)
    clf.fit(X_train, y_train)
    return clf

In [69]:
def get_prepared_data(X, y, target, sensitive):
    """
    Prepare the data for training the LFR model.

    Args:
        X (DataFrame): The input features.
        y (Series): The target labels.
        target (str): The name of the target column.
        sensitive (str): The name of the sensitive attribute column.

    Returns:
        tuple: A tuple containing the prepared dataset, input features, and target labels.
    """
    X_train = X.copy()
    y_train = y.copy()
    X_train[target] = y_train
    train_dataset = DataFrame(X_train)
    train_dataset.targets = target
    train_dataset.sensitive = sensitive
    X_train.drop(columns=[target], inplace=True)
    return train_dataset, X_train, y_train

In [70]:
np.random.seed(42)
torch.manual_seed(42)

EPOCHS = 100
BATCH_SIZE = 128
TARGET = "income"
SENSITIVE = "sex"

# Prepare dataset
X, y = prepare_dataset()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.35, random_state=42
)

In [71]:
lfr_train_dataset, X_train_copy, y_train_copy = get_prepared_data(
    X_train, y_train, TARGET, SENSITIVE
)

lfr = LFR(
    input_dim=X_train.shape[1],
    latent_dim=8,
    output_dim=X_train.shape[1],
    alpha_z=1.0,
    alpha_x=1.0,
    alpha_y=1.0,
)

lfr.fit(lfr_train_dataset, epochs=EPOCHS, batch_size=BATCH_SIZE)

In [72]:
X_train_transformed = lfr.transform(X_train)
X_test_transformed = lfr.transform(X_test)

In [73]:
clf_original = train_classifier(X_train, y_train)
clf_transformed = train_classifier(X_train_transformed, y_train)

In [74]:
y_pred_original = clf_original.predict(X_test)
y_pred_transformed = clf_transformed.predict(X_test_transformed)

acc_original = accuracy_score(y_test, y_pred_original)
acc_transformed = accuracy_score(y_test, y_pred_transformed)
print(f"Accuratezza originale: {acc_original:.4f}")
print(f"Accuratezza trasformata: {acc_transformed:.4f}")

Accuratezza originale: 0.8240
Accuratezza trasformata: 0.8208


In [75]:
spd_original, di_original = evaluate_fairness(X_test.copy(), y_pred_original)
spd_transformed, di_transformed = evaluate_fairness(X_test.copy(), y_pred_transformed)

print("Original Data: SPD:", spd_original, "DI:", di_original, "\n")
print("Transformed Data: SPD:", spd_transformed, "DI:", di_transformed)

Original Data: SPD: {(income=0, sex=0): 0.1681490872919421, (income=0, sex=1): -0.1681490872919421, (income=1, sex=0): -0.1681490872919421, (income=1, sex=1): 0.1681490872919421} DI: {(income=0, sex=0): 0.8250140816862976, (income=0, sex=1): 1.2121005231280875, (income=1, sex=0): 5.303707720688356, (income=1, sex=1): 0.18854734322920272} 

Transformed Data: SPD: {(income=0, sex=0): 0.07976824072217614, (income=0, sex=1): -0.07976824072217614, (income=1, sex=0): -0.07976824072217617, (income=1, sex=1): 0.07976824072217617} DI: {(income=0, sex=0): 0.9119574312775048, (income=0, sex=1): 1.0965424105368182, (income=1, sex=0): 1.848769932178661, (income=1, sex=1): 0.5409001858990435}


In [76]:
for key in spd_original:
    improvement_spd = abs(spd_original[key]) - abs(spd_transformed[key])
    improvement_di = abs(di_original[key] - 1) - abs(di_transformed[key] - 1)

    print(f"Miglioramento per {key}:")
    print(f"  SPD: {improvement_spd:.4f} ({'+' if improvement_spd > 0 else '-'})")
    print(f"  DI: {improvement_di:.4f} ({'+' if improvement_di > 0 else '-'})")

Miglioramento per (income=0, sex=0):
  SPD: 0.0884 (+)
  DI: 0.0869 (+)
Miglioramento per (income=0, sex=1):
  SPD: 0.0884 (+)
  DI: 0.1156 (+)
Miglioramento per (income=1, sex=0):
  SPD: 0.0884 (+)
  DI: 3.4549 (+)
Miglioramento per (income=1, sex=1):
  SPD: 0.0884 (+)
  DI: 0.3524 (+)
